# Courseware環境構築マニュアル（KVM編）
<HR>

このマニュアルでは、同一ホスト上にKVMを配置してCourseware環境をデプロイする手順を説明します。<br>
同一ホスト上に下記のKVMを配置します。

  + Hub
  + Node1
  + Node2


## 0. 前提条件
下記のパッケージをインストールして下さい。
* expect
* qemu-kvm
* bridge-utils
* ssh-keygen

## 1. bredgeを作成する
```
$ sudo brctl addbr jhub-br0
$ sudo ip link set jhub-br0 up
$ sudo bash -c 'echo "allow jhub-br0" >>/etc/qemu-kvm/bridge.conf'
```

## 2. KVMイメージを配置する
**⭐️** 157.1.207.226:/mnt/build-resources/ubuntu-image-resources.tar.gz
をホームディレクトリに転送して解凍して下さい。

## 3. jupyter-platform-devをgit cloneする
任意のディレクトリにjupyter-platform-devのcloneを作成します。
```
$ cd CLONE_DIR
$ git clone git@github.com:axsh/jupyter-platform-dev.git
```

## 4. JupyterHubのビルドディレクトリ環境を作る
BUILD_DIRには、任意のディレクトリ名を指定します。
```
$ cd CLONE_DIR/jupyter-platform-dev
$ ./ind-steps/build-jh-environment/toplevel-kvm-tap-build.sh-new BUILD_DIR BRIDGE
```

## 5. デプロイするイメージをビルドする
90分程度かかります。
```
$ cd BUILD_DIR
$ time ./toplevel-kvm-tap-build.sh do > ../build.log 2>&1
```

## 6. ビルド状況を確認する
```
$ ./toplevel-kvm-tap-build.sh check
```
* 下記以外が（done) になっていることを確認します。
  + ** Step: Install ansible build dependencies (NOT DONE)
  + ** Step: Send "sudo shutdown -h now" via ssh (NOT DONE)
  + ** Step: Wait for KVM to exit (NOT DONE)

## 7. コース用スナップショットを作成する
```
$ cd CLONE_DIR/jupyter-platform-dev  
$ ./ind-steps/build-jh-environment/snapshot-whole-environment.sh BUILD_DIR do guest
```

## 8. コース用のディレクトリ環境を作成して、スナップショットを配置する
COURSE_DIRには任意のディレクトリ名を指定します。
```
$ ./bin/coursectl deploy BUILD_DIR COURSE_DIR BRIDGE
```

実行終了時に、ポートフォワードに関する情報が表示されます。
```
** DOING STEP: Output port forwarding hint script
inet 157.1.207.226
sudo socat TCP-LISTEN:443,fork,reuseaddr TCP:127.0.0.1:6990
```

## 9. コースを初期化する
```
$ ./bin/coursectl new COURSE_DIR MAIL_ADDRESS ACCESS_POINT
```
* MAIL_ADDRESSにはコース管理者（先生）のメールアドレスを指定します。このアドレスから内部で利用するユーザIDを生成します。
* ACCESS_POINTには、このコースのアクセスポイントを指定します。Coursewareが稼働しているホストのホスト名またはFQDNを指定します。
* 実行完了後、下記のフォーマットでコース管理者のログインパスワードが表示されます。メールで直接送付する設定も用意しようと考えています。
  + admin password: nQ8nv]E3wz
* コース管理者は、メールアドレスとこのパスワードでログインできます。

## 10. ポートフォワードを設定する
「8. コース用のディレクトリ環境を作成して、スナップショットを配置する」の手順で取得したポートフォワード情報にしたがって、ポートフォワードを設定します。

## 11. コースにアクセスする
https://ACCESS_POINT/login

* Email Address: コース管理者のメールアドレス
* Password: "/bin/coursectl new"実行時に返されたパスワード